In [1]:
import cPickle
import datetime
import os

In [2]:
# load data from pickle
f = open( 'data.pkl', 'r' )

classes = cPickle.load( f )
chars = cPickle.load( f )
char_indices = cPickle.load( f )
indices_char = cPickle.load( f )

maxlen = cPickle.load( f )
step = cPickle.load( f )

X = cPickle.load( f )
y = cPickle.load( f )

f.close()

## Data

All sequences in `X` appear to be `99 + 12 = 111` characters in length.  The ground truth `y` is length `99`.

In [3]:
X.shape

(3066887, 20, 111)

In [4]:
y.shape

(3066887, 99)

In [5]:
len(classes)

12

In [6]:
len(chars)

99

In [7]:
(maxlen, step)

(20, 1)

In [8]:
classes

['c34.0',
 'c34.1',
 'c34.2',
 'c34.3',
 'c34.9',
 'c50.1',
 'c50.2',
 'c50.3',
 'c50.4',
 'c50.5',
 'c50.8',
 'c50.9']

## Conversion to TF


In [9]:
import itertools
import tensorflow as tf

In [10]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bool_features(value):
    return tf.trainFeature(int64_list)

In [11]:
def make_example(seq, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'length': _int64_feature(len(seq)),
        'input': tf.train.Feature(int64_list=tf.train.Int64List(value=seq.astype(int).reshape(-1).tolist())),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=label.astype(int).tolist())),
    }))

In [12]:
with open("tf-records.data", "w") as output:
    writer = tf.python_io.TFRecordWriter(output.name)
    for input, label in itertools.izip(X,y):
        writer.write(make_example(input, label).SerializeToString())
    writer.close()

NameError: name 'fp' is not defined

## Serialize and Deserialize TFRecord / Example

This sections just shows some basic manipulation of TFRecords.

In [22]:
example = make_example(X[10], y[10])

In [125]:
def extract_features(serialized_example):
    return tf.parse_single_example(
        serialized=serialized_example,
        features={
            'label': tf.FixedLenFeature([99], dtype=tf.int64),
            'input': tf.FixedLenFeature([20,111], dtype=tf.int64),
        }
    )

In [126]:
features = extract_features(example.SerializeToString())

In [127]:
features

{'input': <tf.Tensor 'ParseSingleExample_30/Squeeze_input:0' shape=(20, 111) dtype=int64>,
 'label': <tf.Tensor 'ParseSingleExample_30/Squeeze_label:0' shape=(99,) dtype=int64>}

In [128]:
#tf.contrib.learn.run_n
tf.contrib.learn.run_n(features, n=1, feed_dict=None, restore_checkpoint_path=None)

[{'input': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ..., 
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]),
  'label': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0])}]

In [130]:
def read_and_decode(serialized_example):
    features = extract_features(serialized_example)
    input = tf.cast(features['input'], tf.float32)
    label = tf.cast(features['label'], tf.float32)
    return input, label

In [131]:
input, label = read_and_decode(example.SerializeToString())
x = tf.contrib.learn.run_n({'input': input}, n=1, feed_dict=None, restore_checkpoint_path=None)

In [132]:
x[0]['input'][0].shape

(111,)